In [34]:
import pandas as pd
import numpy as np

In [35]:
TEAMS = {"MIBR": 0, 
         "Leviatán": 1, 
         "Sentinels": 2, 
         "NRG Esports": 3, 
         "FURIA": 4, 
         "100 Thieves": 5, 
         "LOUD" : 6, 
         "Evil Geniuses" : 7, 
         "G2 Esports" : 8, 
         "Cloud9" : 9, 
         "KRÜ Esports" : 10, 
         "Titan Esports Club" : 11, 
         "JDG Esports" : 12, 
         "All Gamers" : 13, 
         "TYLOO" : 14, 
         "Bilibili Gaming" : 15, 
         "Wolves Esports" : 16, 
         "Dragon Ranger Gaming" : 17, 
         "Nova Esports" : 18, 
         "FunPlus Phoenix" : 19, 
         "Trace Esports" : 20, 
         "EDward Gaming" : 21, 
         "ZETA DIVISION" : 22, 
         "DRX" : 23, 
         "Team Secret" : 24, 
         "BLEED" : 25, 
         "T1" : 26, 
         "Gen.G" : 27, 
         "Paper Rex" : 28, 
         "Talon Esports" : 29, 
         "Rex Regum Qeon" : 30, 
         "DetonatioN FocusMe" : 31, 
         "Global Esports" : 32, 
         "FUT Esports" : 33, 
         "KOI" : 34, 
         "BBL Esports" : 35, 
         "FNATIC" : 36, 
         "Team Heretics" : 37, 
         "Natus Vincere" : 38, 
         "GIANTX" : 39, 
         "Gentle Mates" : 40, 
         "Team Vitality" : 41, 
         "Team Liquid" : 42, 
         "Karmine Corp": 43}

MAPS = {"Bind" : 0, 
        "Haven" : 1, 
        "Split" : 2, 
        "Ascent" : 3,
        "Icebox" : 4, 
        "Breeze" : 5, 
        "Fracture" : 6, 
        "Abyss" : 7, 
        "Lotus" : 8, 
        "Sunset" : 9, 
        "Pearl" : 10}

In [36]:
data = pd.read_csv("Dataset/maps_scores.csv")
data = data[["Map", "Team A", "Team B", "Team A Score", "Team B Score"]]
data["Team A Delta"] = data["Team A Score"] - data["Team B Score"]
data["Team B Delta"] = data["Team B Score"] - data["Team A Score"]
data.loc[data["Team A Delta"] < 0, "Team A Win Chance"] = 0
data.loc[data["Team A Delta"] > 0, "Team A Win Chance"] = 100
data

,Map,Team A,Team B,Team A Score,Team B Score,Team A Delta,Team B Delta,Team A Win Chance
0,Ascent,MIBR,Leviatán,9,13,-4,4,0.0
1,Icebox,MIBR,Leviatán,7,13,-6,6,0.0
2,Lotus,Sentinels,NRG Esports,13,8,5,-5,100.0
3,Sunset,Sentinels,NRG Esports,14,12,2,-2,100.0
4,Icebox,FURIA,100 Thieves,14,12,2,-2,100.0
...,...,...,...,...,...,...,...,...
1013,Lotus,DRX,Gen.G,8,13,-5,5,0.0
1014,Ascent,Paper Rex,Gen.G,5,13,-8,8,0.0
1015,Split,Paper Rex,Gen.G,13,4,9,-9,100.0
1016,Lotus,Paper Rex,Gen.G,11,13,-2,2,0.0


In [37]:
ACS = pd.read_csv("Dataset/overview.csv")
ACS = ACS[["Match Type", "Map", "Match Name", "Team", "Average Combat Score"]]
ACS.groupby("Team")['Average Combat Score']

In [38]:
TeamAcs = ACS.groupby(["Team", "Map"])['Average Combat Score'].mean().reset_index()
TeamAcs

,Team,Map,Average Combat Score
0,100 Thieves,Abyss,200.400000
1,100 Thieves,Ascent,219.828571
2,100 Thieves,Bind,207.342857
3,100 Thieves,Breeze,226.666667
4,100 Thieves,Haven,197.266667
...,...,...,...
346,ZETA DIVISION,Haven,209.400000
347,ZETA DIVISION,Icebox,183.900000
348,ZETA DIVISION,Lotus,192.914286
349,ZETA DIVISION,Split,171.800000


In [39]:
merged_data = data.merge(TeamAcs, left_on=["Team A", "Map"], right_on=["Team", "Map"], how="left")
merged_data = merged_data.merge(TeamAcs, left_on=["Team B", "Map"], right_on=["Team", "Map"], how="left")
merged_data = merged_data.rename(columns={"Average Combat Score_x" : "Team A ACS", "Average Combat Score_y" : "Team B ACS"})
merged_data.drop(columns=["Team_x", "Team_y"], inplace=True)
merged_data = merged_data.replace({"Team A" : TEAMS, "Team B" : TEAMS, "Map" : MAPS})
merged_data["Team B"]

/var/folders/bc/3xp67nwd5371m_f3nw5xqvrh0000gn/T/ipykernel_56735/934029576.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data = merged_data.replace({"Team A" : TEAMS, "Team B" : TEAMS, "Map" : MAPS})


0        1
1        1
2        3
3        3
4        5
        ..
1013    27
1014    27
1015    27
1016    27
1017    27
Name: Team B, Length: 1018, dtype: int64